In [1]:
# TODO: change rupees to euro, standarize os(windows/linux/macos), add resolution_category (FHD, HD, 4K), seprate VRAM from gpu
# reorder model_name, brand, cpu, cpu_cores, cpu_threads, ram_gb, ssd_gb, hdd_gb, os, gpu, screen_size_in, resolution_w, resolution_h, resolution_type, spec_score, price_euro
import pandas as pd

df_raw = pd.read_csv("../data/modern_laptops.csv",index_col=0)
df_raw.head()

,model_name,brand,processor_name,ram(GB),ssd(GB),Hard Disk(GB),Operating System,graphics,screen_size(inches),resolution (pixels),no_of_cores,no_of_threads,spec_score,price_inIndianRupees
0,Lenovo V15 ITL G2 82KBA033IH Laptop,Lenovo,11th Gen Core i3,8,512,0,Windows,Intel Integrated UHD,15.6,1920 x 1080,2,4,62,33921
1,HP Pavilion 15-ec2004AX Gaming Laptop,HP,AMD Ryzen 5 5600H,8,512,0,Windows,4 GB NVIDIA GeForce GTX 1650,15.6,1920 x 1080,6,12,66,56150
2,Lenovo V15 82KBA03HIH Laptop,Lenovo,11th Gen Core i3,8,256,1000,Windows,Intel Integrated UHD,15.6,1920 x 1080,2,4,66,35499
3,Asus Vivobook 16X 2022 M1603QA-MB502WS Laptop,Asus,Ryzen 5-5600H,8,512,0,Windows,AMD Radeon Vega 7,16.0,1200 x 1920,6,12,66,48990
4,HP 15s-fr4000TU Laptop,HP,11th Gen Core i5,8,512,0,Windows,Intel Integrated Iris Xe,15.6,1920 x 1080,4,8,63,52990


In [2]:
df = df_raw.rename(
    columns={
        "processor_name": "cpu",
        "ram(GB)": "ram",
        "ssd(GB)": "ssd_gb",
        "Hard Disk(GB)": "hdd_gb",
        "Operating System": "os",
        "graphics": "gpu",
        "screen_size(inches)": "screen_size_in",
        "no_of_cores": "cpu_cores",
        "no_of_threads": "cpu_threads",
    }
)


In [3]:
df = df.dropna(subset=["resolution (pixels)"])
df = df[df["gpu"].str.lower() != "missing"].reset_index(drop=True)

In [4]:
df["resolution_w"] = (
    df["resolution (pixels)"]
    .str.split("x")
    .str[0]
    .str.strip()
    .astype(int)
)
df["resolution_h"] = (
    df["resolution (pixels)"]
    .str.split("x")
    .str[1]
    .str.strip()
    .astype(int)
)

In [5]:
mask = df["resolution_w"] < df["resolution_h"]
df.loc[mask, ["resolution_w", "resolution_h"]] = df.loc[mask, ["resolution_h", "resolution_w"]].values

In [6]:
def get_resolution_type(w, h):
    if w >= 3840 and h >= 2160: return "4K"
    elif w >= 2560 and h >= 1440: return "QHD"
    elif w >= 1920 and h >= 1080: return "FHD"
    else: return "HD"

df["resolution_type"] = df.apply(lambda row: get_resolution_type(row["resolution_w"], row["resolution_h"]), axis=1)


In [7]:
df["os"] = df["os"].replace({"Ubuntu": "Linux", "DOS": "Linux"})


In [8]:
import re

def normalize_cpu(cpu: str) -> str:
    cpu = cpu.strip()
    
    # Already normalized
    if cpu.startswith(("Intel ", "AMD ", "Apple ")):
        return cpu
    
    # Handle Intel generational format: "11th Gen Core i5" -> "Intel Core i5 11th Gen"
    gen_match = re.match(r'(\d+)(?:th|st|nd|rd)\s+Gen\s+(Core i[3579])', cpu, re.IGNORECASE)
    if gen_match:
        gen = f"{gen_match.group(1)}th Gen"
        tier = gen_match.group(2)
        # Ensure 'i' is lowercase: "Core I5" -> "Core i5"
        tier = re.sub(r'([iI])(\d)', r'i\2', tier)
        return f"Intel {tier} {gen}"
    
   # Intel: Celeron or Pentium
    if re.search(r'\b[Cc]eleron\b|\b[Pp]entium\b', cpu):
        return "Intel " + re.sub(r'^(Intel\s*)?', '', cpu)
    
    # AMD: Ryzen or Athlon 
    if re.search(r'\b[Rr]yzen\b|\b[Aa]thlon\b', cpu):
        return "AMD " + re.sub(r'^(AMD\s*)?', '', cpu)

 
    
    # If none match, return as is
    return cpu

In [9]:
df['cpu'] = df['cpu'].apply(normalize_cpu)

In [10]:
df["price_euro"] = df["price_inIndianRupees"] * 0.0097
df["price_euro"] = df["price_euro"].round(2)

In [11]:
import numpy as np
import re

def extract_vram(gpu_str):
    if pd.isna(gpu_str):
        return pd.Series([np.nan, np.nan])
    
    match = re.match(r"^(\d+\s*GB)\s+(.*)", gpu_str)
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([np.nan, gpu_str])

df[["gpu_vram", "gpu"]] = df["gpu"].apply(extract_vram)

In [12]:
df.drop(columns=["price_inIndianRupees","resolution (pixels)"])

,model_name,brand,cpu,ram,ssd_gb,hdd_gb,os,gpu,screen_size_in,cpu_cores,cpu_threads,spec_score,resolution_w,resolution_h,resolution_type,price_euro,gpu_vram
0,Lenovo V15 ITL G2 82KBA033IH Laptop,Lenovo,Intel Core i3 11th Gen,8,512,0,Windows,Intel Integrated UHD,15.6,2,4,62,1920,1080,FHD,329.03,NaN
1,HP Pavilion 15-ec2004AX Gaming Laptop,HP,AMD Ryzen 5 5600H,8,512,0,Windows,NVIDIA GeForce GTX 1650,15.6,6,12,66,1920,1080,FHD,544.66,4 GB
2,Lenovo V15 82KBA03HIH Laptop,Lenovo,Intel Core i3 11th Gen,8,256,1000,Windows,Intel Integrated UHD,15.6,2,4,66,1920,1080,FHD,344.34,NaN
3,Asus Vivobook 16X 2022 M1603QA-MB502WS Laptop,Asus,AMD Ryzen 5-5600H,8,512,0,Windows,AMD Radeon Vega 7,16.0,6,12,66,1920,1200,FHD,475.20,NaN
4,HP 15s-fr4000TU Laptop,HP,Intel Core i5 11th Gen,8,512,0,Windows,Intel Integrated Iris Xe,15.6,4,8,63,1920,1080,FHD,514.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,Fujitsu UH-X 4ZR1F38024 Laptop,Fujitsu,Intel Core i7 11th Gen,16,512,0,Windows,Intel Iris Xe Graphics,13.3,4,8,63,1920,1080,FHD,824.40,NaN
1001,Asus BR1100CKA-GJ0722W Laptop,Asus,Intel Celeron N4500,4,128,0,Windows,Intel UHD Graphics,11.6,2,0,45,1366,768,HD,223.00,NaN
1002,Lenovo Yoga 6 13ALC6 82ND00DPIN Laptop,Lenovo,AMD Ryzen 7 5700U,16,1000,0,Windows,AMD Radeon AMD,13.3,8,16,68,1920,1080,FHD,902.09,NaN
1003,Asus BR1100FKA-BP1104W Laptop,Asus,Intel Celeron N4500,4,128,0,Windows,Intel Integrated UHD Graphics,11.6,2,0,49,1366,768,HD,290.90,NaN


In [13]:
df = df[[
    "model_name", "brand", "cpu", "cpu_cores", "cpu_threads",
    "ram", "ssd_gb", "hdd_gb", "os", "gpu", "gpu_vram",
    "screen_size_in", "resolution_w", "resolution_h",
    "resolution_type", "spec_score", "price_euro"
]]


In [14]:
df.head()

,model_name,brand,cpu,cpu_cores,cpu_threads,ram,ssd_gb,hdd_gb,os,gpu,gpu_vram,screen_size_in,resolution_w,resolution_h,resolution_type,spec_score,price_euro
0,Lenovo V15 ITL G2 82KBA033IH Laptop,Lenovo,Intel Core i3 11th Gen,2,4,8,512,0,Windows,Intel Integrated UHD,NaN,15.6,1920,1080,FHD,62,329.03
1,HP Pavilion 15-ec2004AX Gaming Laptop,HP,AMD Ryzen 5 5600H,6,12,8,512,0,Windows,NVIDIA GeForce GTX 1650,4 GB,15.6,1920,1080,FHD,66,544.66
2,Lenovo V15 82KBA03HIH Laptop,Lenovo,Intel Core i3 11th Gen,2,4,8,256,1000,Windows,Intel Integrated UHD,NaN,15.6,1920,1080,FHD,66,344.34
3,Asus Vivobook 16X 2022 M1603QA-MB502WS Laptop,Asus,AMD Ryzen 5-5600H,6,12,8,512,0,Windows,AMD Radeon Vega 7,NaN,16.0,1920,1200,FHD,66,475.20
4,HP 15s-fr4000TU Laptop,HP,Intel Core i5 11th Gen,4,8,8,512,0,Windows,Intel Integrated Iris Xe,NaN,15.6,1920,1080,FHD,63,514.00


In [15]:
df.to_csv("../data/laptops_cleaned.csv",index=False)
